# Definitions

In [1]:
import json
import pandas as pd
import numpy as np
import requests
from skbio.alignment import global_pairwise_align_protein
from skbio.sequence import Protein, Sequence
from skbio.sequence.distance import hamming
from ete3 import Tree, TreeStyle, NodeStyle
import matplotlib.pyplot as plt

In [2]:
pam30 = pd.read_csv('../datasets/pam30.txt', sep="\s+", skiprows=9, index_col=0).to_dict()

# Representants

In [30]:
sequences_extended_filtered = pd.read_csv('../datasets/sequences_extended_filtered.csv').set_index('Accession')

In [31]:
with open(f'clusters.json', 'r') as f:
    representants = [
        sequences_extended_filtered.loc[accessions].sort_values('Collection_Date').iloc[0].name
        for _, accessions in json.loads(f.read()).items()
    ]
with open(f'representants.fasta', 'w') as f:
    response = requests.post(f'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=protein&rettype=fasta&retmode=text', {'id': ",".join(representants)})
    f.write(response.text)

# Scores and distances

In [32]:
%%time
proteins = sequences_extended_filtered.loc[representants]['Sequence'].apply(lambda s: Protein(s.replace('\n', '')))
alignments = pd.DataFrame([[global_pairwise_align_protein(p1, p2, substitution_matrix=pam30) for p1 in proteins[:i+1]] for i, p2 in enumerate(proteins)], index=representants, columns=representants)

CPU times: user 38min 30s, sys: 1.46 s, total: 38min 32s
Wall time: 38min 35s


In [33]:
alignments.to_pickle('alignments.p')

In [34]:
scores = alignments.applymap(lambda x: x[1] if x else x)
scores.to_csv('scores.csv')
print(scores)
distances = alignments.applymap(lambda x: hamming(x[0][0], x[0][1]) * len(x[0][0]) if x else x)
distances.to_csv('distances.csv')
print(distances)

          QOQ09568  QLG76049  QKV49386  QNO80951  QJT72902  QOF16085  \
QOQ09568    9685.0       NaN       NaN       NaN       NaN       NaN   
QLG76049    9632.0    9684.0       NaN       NaN       NaN       NaN   
QKV49386    9615.0    9622.0    9686.0       NaN       NaN       NaN   
QNO80951    9650.0    9623.0    9606.0    9690.0       NaN       NaN   
QJT72902    9640.0    9627.0    9610.0    9631.0    9683.0       NaN   
QOF16085    9644.0    9631.0    9614.0    9635.0    9639.0    9688.0   
QKF95522    9608.0    9615.0    9679.0    9599.0    9603.0    9607.0   
QJX45344    9647.0    9634.0    9617.0    9638.0    9642.0    9646.0   
QHD43416    9655.0    9662.0    9645.0    9646.0    9650.0    9654.0   

          QKF95522  QJX45344  QHD43416  
QOQ09568       NaN       NaN       NaN  
QLG76049       NaN       NaN       NaN  
QKV49386       NaN       NaN       NaN  
QNO80951       NaN       NaN       NaN  
QJT72902       NaN       NaN       NaN  
QOF16085       NaN       NaN     

# UPGMA

In [35]:
def upgma(distances):
    assert distances.index.tolist() == distances.columns.tolist()
    d = distances.to_numpy()
    d[d == 0] = np.nan
    d = np.tril(d) + np.tril(d).T
    l = [(s,) for s in distances.index]
    t = [Tree(f'{s};') for s in distances.index]
    while len(l) > 1:
        r, c = np.unravel_index(np.nanargmin(d, axis=None), d.shape)
        dist = d[r, c] / 2
        n = (d[r] * len(t[r]) + d[c] * len(t[c])) / (len(t[r]) + len(t[c]))
        d[r,:] = n
        d[:,r] = n.T
        d = np.delete(d, c, 0)
        d = np.delete(d, c, 1)

        l[r] = (l[r] + l.pop(c),)

        l_subtree = t[r]
        r_subtree = t.pop(c)
        tree = Tree()
        tree.add_child(l_subtree, dist=dist-l_subtree.get_distance(next(l_subtree.iter_leaves())))
        tree.add_child(r_subtree, dist=dist-r_subtree.get_distance(next(r_subtree.iter_leaves())))
        t[r] = tree
    return t[0]
       

In [40]:
%%capture
distances = pd.read_csv('distances.csv', index_col=0)
t = upgma(distances)
nstyle = NodeStyle()
nstyle["size"] = 0
for n in t.traverse():
   n.set_style(nstyle)
t.render('upgma.png', dpi=120, w=1200)

# Geography

In [5]:
with open('clusters.json', 'r') as f:
    clusters = json.loads(f.read())

In [6]:
sequences_extended_filtered = pd.read_csv('../datasets/sequences_extended_filtered.csv').set_index('Accession')
geo = {i: sequences_extended_filtered.loc[accessions, 'Geo_Location'].apply(lambda x: x.split(':')[0]).value_counts() for i, accessions in clusters.items()}
for i, vc in geo.items():
    print('='*40)
    print(f'Value counts for cluster #{i}')
    print()
    print(vc)
    print()
    print()
    print()

Value counts for cluster #0

Australia     161
USA             5
Tunisia         1
India           1
Bangladesh      1
Name: Geo_Location, dtype: int64



Value counts for cluster #1

USA             83
Australia       12
India            6
China            5
Hong Kong        3
Greece           2
Thailand         2
Iran             2
Malaysia         1
Germany          1
France           1
Sweden           1
Peru             1
Egypt            1
South Korea      1
Sierra Leone     1
Taiwan           1
Turkey           1
Jordan           1
Poland           1
Finland          1
Name: Geo_Location, dtype: int64



Value counts for cluster #2

USA            17
India           2
Taiwan          2
France          2
Australia       2
Venezuela       1
Israel          1
South Korea     1
Iran            1
Georgia         1
Bangladesh      1
Hong Kong       1
Name: Geo_Location, dtype: int64



Value counts for cluster #3

Australia     31
USA           13
India          7
Egypt          1
Ira

# Mutation rate

In [7]:
with open(f'clusters.json', 'r') as f:
    last_representants = [
        sequences_extended_filtered.loc[accessions].sort_values('Collection_Date', ascending=False).iloc[0].name
        for _, accessions in json.loads(f.read()).items()
    ]
    last_proteins = sequences_extended_filtered.loc[last_representants]['Sequence'].apply(lambda s: Protein(s.replace('\n', '')))
sequences_extended = pd.read_csv('../datasets/sequences_extended.csv').set_index('Accession')
sequences_extended['Collection_Date'] = pd.to_datetime(sequences_extended['Collection_Date'])
YP_009724390 = Protein(sequences_extended.loc['YP_009724390', 'Sequence'].replace('\n', ''))
mutation_rates = [hamming(*global_pairwise_align_protein(protein, YP_009724390, substitution_matrix=pam30)[0]) * len(YP_009724390) / ((sequences_extended.loc[accession, 'Collection_Date'] - sequences_extended.loc['YP_009724390', 'Collection_Date']).days) for protein, accession in zip(last_proteins, last_representants)]

In [8]:
mutation_rates

[0.008902077151335312,
 0.004329004329004329,
 0.013468013468013467,
 0.014234875444839857,
 0.00911854103343465,
 0.01090909090909091,
 0.015209125475285171,
 0.009202453987730062,
 0.006097560975609756]